In [1]:
from ipywidgets import IntProgress
from IPython.display import display
import time

import os
import sys
module_path = os.path.abspath(os.path.join('./covid_vulnerabilidad_mex'))
if module_path not in sys.path:
    sys.path.append(module_path)

from covid19_vulnerabilidad_mex.datos import *
from covid19_vulnerabilidad_mex.vulnerabilidad import *

import pandas as pd
import geopandas as gpd

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime
from datetime import timedelta, date, datetime

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

from pygifsicle import optimize
from pathlib import Path


In [2]:
%load_ext autoreload
%autoreload 2

## Nuevos datos de comorbilidades

Del trabajo de INEGI de estimación en áreas pequeñas

In [ ]:
comorbilidades = pd.read_excel("datos/municipios/a_peq_prev_2018.xlsx",
                               engine='openpyxl',
                               skiprows=2,
                               nrows=12450,
                               usecols='A:I',
                               dtype={'Identificador único del municipio':str,
                                      'Clave de entidad federativa':str,
                                      'Clave de municipio o delegación':str}
                            ).rename({'Identificador único del municipio': 'cvegeo',
                                      'Porcentaje de población de 20 años y más con obesidad.':'pct_obesidad', 
                                      'Porcentaje de población de 20 años y más con diagnóstico previo de hipertensión.':'pct_hipertension',
                                      'Porcentaje de población de 20 años y más con diagnóstico previo de diabetes.':'pct_diabetes',
                                      'Clave de entidad federativa': 'cve_ent'},
                                      axis=1)
comorbilidades = comorbilidades[comorbilidades['Municipio o delegación'] != 'Total']
comorbilidades = comorbilidades[comorbilidades['Estimador'] == 'Valor']
comorbilidades = comorbilidades[['cve_ent', 'cvegeo', 'pct_obesidad', 'pct_hipertension', 'pct_diabetes']]
comorbilidades


,cve_ent,cvegeo,pct_obesidad,pct_hipertension,pct_diabetes
10,01,01001,31.486541,14.942242,7.495861
15,01,01002,32.282284,15.320425,7.953634
20,01,01003,40.004293,13.751906,9.172624
25,01,01004,32.596450,16.431493,7.383116
30,01,01005,34.731715,12.356755,6.745819
...,...,...,...,...,...
12425,32,32054,31.075955,16.920279,9.895491
12430,32,32055,30.526273,21.746589,12.050064
12435,32,32056,36.822130,20.010453,11.814172
12440,32,32057,32.477646,15.941678,8.089966


In [ ]:
comorbilidades['cve_ent'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'],
      dtype=object)

In [ ]:
variables_municipales = pd.read_csv('datos/municipios/indicadores_original.csv', index_col=False,
                                    dtype={'cvegeo': str, 'entidad_cv': str,
                                            'municipio_cvegeo': str, 'entidad_cvegeo': str},
                                    encoding='iso8859_2')
arregla_cvegeo(variables_municipales)
variables_municipales.head()

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,hospitales_sme,hospitales_semar,hospitales_sedena,hospitales_pemex,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,0,0,0,0,8,1228,17,43,1271,961977
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,0,0,0,0,0,0,0,0,0,50864
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,0,0,0,0,1,46,1,0,46,60760
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,0,0,0,0,0,0,0,0,0,16918
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,0,0,0,0,1,297,0,0,297,130184


In [ ]:
variables_municipales.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'tasa_diabetes', 'tasa_cardiacas',
       'tasa_cancer', 'tasa_pulmo

In [ ]:
variables_nuevas = variables_municipales.merge(comorbilidades, on='cvegeo')
variables_nuevas.drop(columns=[c for c in variables_nuevas.columns if (c.startswith('grado') or c.startswith('tasa'))], inplace=True)
variables_nuevas

KeyboardInterrupt: 

In [ ]:
variables_nuevas.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'hospitales_imss', 'hospitales_issste',
       'hospitales_ssa', 'ho

## Nuevas variables de hospitales

* Eliminar división por sector (isste, imss, etc)
* Sólo: total_hospitales_publicos y privados y los respectivos números de camas
* Agregados por estado porque no tiene mucho sentido por municipio
* Las camas de hospitales privados creo que tienen mucho error, las voy a eliminar

In [ ]:
variables_nuevas.drop(columns=['hospitales_imss',
                               'hospitales_issste', 'hospitales_ssa', 'hospitales_sme',
                               'hospitales_semar', 'hospitales_sedena', 'hospitales_pemex', 'total_camas'], inplace=True)
variables_nuevas["total_camas_publicos_estatal"] = (variables_nuevas[['nom_ent', 'total_camas_publicos']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_camas_privados_estatal"] = (variables_nuevas[['nom_ent', 'total_de_camas_privados']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_hospitales_publicos_estatal"] = (variables_nuevas[['nom_ent', 'total_hospitales_publicos']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))
variables_nuevas["total_hospitales_privados_estatal"] = (variables_nuevas[['nom_ent', 'hospitales_privados']]
                                                    .groupby('nom_ent')
                                                    .transform('sum'))                                                 
variables_nuevas.drop(columns=['total_camas_publicos', 'total_de_camas_privados',
                               'total_hospitales_publicos', 'hospitales_privados'], inplace=True)
variables_nuevas.rename({"total_camas_publicos_estatal": "total_camas_publicos",
                         "total_camas_privados_estatal": "total_camas_privados",
                         "total_hospitales_publicos_estatal": "total_hospitales_publicos",
                         "total_hospitales_privados_estatal": "total_hospitales_privados"}, axis=1, inplace=True)
variables_nuevas.drop(columns="total_camas_privados")
variables_nuevas

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,pob01,pob2020,cve_ent,pct_obesidad,pct_hipertension,pct_diabetes,total_camas_publicos,total_camas_privados,total_hospitales_publicos,total_hospitales_privados
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,797010,961977,01,31.486541,14.942242,7.495861,1656,43,12,20
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,45492,50864,01,32.282284,15.320425,7.953634,1656,43,12,20
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,54136,60760,01,40.004293,13.751906,9.172624,1656,43,12,20
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,15042,16918,01,32.596450,16.431493,7.383116,1656,43,12,20
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,99590,130184,01,34.731715,12.356755,6.745819,1656,43,12,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,18490,20177,32,31.075955,16.920279,9.895491,1122,0,29,49
2453,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,29395,31804,32,30.526273,21.746589,12.050064,1122,0,29,49
2454,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,138176,155533,32,36.822130,20.010453,11.814172,1122,0,29,49
2455,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,16934,20285,32,32.477646,15.941678,8.089966,1122,0,29,49


In [ ]:
variables_nuevas.columns

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'pob2020', 'pct_obesidad',
       'pct_hipertension', 'pct_diabetes'

In [ ]:
variables_nuevas.to_csv("datos/municipios/indicadores.csv", index=False)

## Recodificación

Vamos a recodificar las variables de forma que siempre un valor más grande implique una mayor vulnerabilidad. Para eso vamos a partir del archivo con las nuevas variables de INEGI y los hospitales agregados por estado

Las variables a reclasificar son:

* Total de hospitales
* Total de camas
* Consultorios. Antes vamos a ponderar por población
* Farmacias. Antes vamos a ponderar por población
* npnv

Para reclasificar vamos a  reescalar la variable (MinMax) y para invertir la variaciónb vamos a restar el valor reescalado a 1

In [3]:
variables_municipales = pd.read_csv('datos/municipios/indicadores_nueva_comorbilidad.csv', index_col=False,
                                    dtype={'cvegeo': str, 'entidad_cv': str,
                                            'municipio_cvegeo': str, 'entidad_cvegeo': str},
                                    encoding='iso8859_2')
arregla_cvegeo(variables_municipales)
variables_municipales.head()

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,pob01,pob2020,cve_ent,pct_obesidad,pct_hipertension,pct_diabetes,total_camas_publicos,total_camas_privados,total_hospitales_publicos,total_hospitales_privados
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,797010,961977,1,31.486541,14.942242,7.495861,1656,43,12,20
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,45492,50864,1,32.282284,15.320425,7.953634,1656,43,12,20
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,54136,60760,1,40.004293,13.751906,9.172624,1656,43,12,20
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,15042,16918,1,32.596450,16.431493,7.383116,1656,43,12,20
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,99590,130184,1,34.731715,12.356755,6.745819,1656,43,12,20


In [4]:
reclasifica = ['total_camas_publicos', 'total_camas_privados', 'total_hospitales_publicos', 'total_hospitales_privados',
               'consultori', 'consulto_1', 'farmacias', 'npnv']
variables_municipales['consultori'] = variables_municipales['consultori'] / variables_municipales['pob2020']
variables_municipales['consulto_1'] = variables_municipales['consulto_1'] / variables_municipales['pob2020'] 
variables_municipales['farmacias'] = variables_municipales['farmacias'] / variables_municipales['pob2020'] 
for columna in reclasifica:
    variables_municipales = rescala_min_max(variables_municipales, columna)
    variables_municipales[columna] = 1 - variables_municipales[columna]
variables_municipales

,cvegeo,nom_ent,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,...,pob01,pob2020,cve_ent,pct_obesidad,pct_hipertension,pct_diabetes,total_camas_publicos,total_camas_privados,total_hospitales_publicos,total_hospitales_privados
0,01001,Aguascalientes,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,...,797010,961977,1,31.486541,14.942242,7.495861,0.959461,0.993091,0.981481,0.965385
1,01002,Aguascalientes,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,...,45492,50864,1,32.282284,15.320425,7.953634,0.959461,0.993091,0.981481,0.965385
2,01003,Aguascalientes,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,...,54136,60760,1,40.004293,13.751906,9.172624,0.959461,0.993091,0.981481,0.965385
3,01004,Aguascalientes,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,...,15042,16918,1,32.596450,16.431493,7.383116,0.959461,0.993091,0.981481,0.965385
4,01005,Aguascalientes,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,...,99590,130184,1,34.731715,12.356755,6.745819,0.959461,0.993091,0.981481,0.965385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,32054,Zacatecas,19155,8.1332,3.6565,42.1699,8.8384,2.5438,8.9568,3.8478,...,18490,20177,32,31.075955,16.920279,9.895491,0.991437,1.000000,0.824074,0.909615
2453,32055,Zacatecas,30240,4.7985,3.4198,53.7212,13.5516,0.9201,4.8484,1.7459,...,29395,31804,32,30.526273,21.746589,12.050064,0.991437,1.000000,0.824074,0.909615
2454,32056,Zacatecas,146147,1.2432,2.1746,23.0304,14.0968,0.2485,1.2004,1.6826,...,138176,155533,32,36.822130,20.010453,11.814172,0.991437,1.000000,0.824074,0.909615
2455,32057,Zacatecas,19413,5.6122,2.5997,42.0488,9.0609,1.1950,6.9448,3.0891,...,16934,20285,32,32.477646,15.941678,8.089966,0.991437,1.000000,0.824074,0.909615


In [5]:
variables_municipales.drop(columns="cve_ent").to_csv("datos/municipios/indicadores_reclasificados.csv", index=False)

## Selección de variables

Aquí vmos a intentar hacer una mejor selección de variables a partir de los indicadores reclasificados

In [4]:
variables_municipales = pd.read_csv('datos/municipios/indicadores_reclasificados.csv', index_col=False,
                                    dtype={'cvegeo': str, 'entidad_cv': str,
                                            'municipio_cvegeo': str, 'entidad_cvegeo': str},
                                    encoding='iso8859_2')
variables_municipales.columns                                   

Index(['cvegeo', 'nom_ent', 'pt_2015', 'an_2015', 'ne614_015', 'bi_2015',
       'sins_15', 'vpt_2015', 'vns_2015', 'vna_2015', 'vnd_2015', 'vne_2015',
       'vnl_2015', 'vnr_2015', 'irs_2015', 'grs_2015', 'lmex_2015',
       'poblacion', 'pobreza', 'pobreza_pob', 'pobreza_e', 'pobreza_e_pob',
       'pobreza_m', 'pobreza_m_pob', 'vul_car', 'vul_car_pob', 'vul_ing',
       'vul_ing_pob', 'npnv', 'npnv_pob', 'ic_rezedu', 'ic_rezedu_pob',
       'ic_asalud', 'ic_asalud_pob', 'ic_segsoc', 'ic_segsoc_pob', 'ic_cv',
       'ic_cv_pob', 'ic_sbv', 'ic_sbv_pob', 'ic_ali', 'ic_ali_pob',
       'carencias', 'carencias_pob', 'carencias3', 'carencias3_pob', 'plb',
       'plb_pob', 'plbm', 'plbm_pob', 'pob_total', 'mayores_65', 'pob_menore',
       'sin_derech', 'sin_dere_1', 'entidad_cv', 'consultori', 'consulto_1',
       'farmacias', 'id', 'municipio_cvegeo', 'nom_mun', 'porc_carencia_salud',
       'entidad_cvegeo', 'pob01', 'pob2020', 'pct_obesidad',
       'pct_hipertension', 'pct_diabetes'

In [ ]:
## Clasificación en dimensiones

A partir del diccionario de datos vamos a construir uno nuevo con las variables que usamos. Este nuevo es el que tendrá la clasificación

In [ ]:
diccionario = pd.read_csv("datos/diccionario_vulnerabilidad.csv")
diccionario = diccionario.replace({'total_de_camas_privados':'total_camas_privados'})
diccionario

,campo,descriptor
0,cvegeo,Clave geografica
1,pt_2000,"población total,2000"
2,pt_2005,"población total,2005"
3,pt_2010,"población total,2010"
4,pt_2015,"población total,2015"
...,...,...
122,tasa_pulmonares,"Enfermedades del sistema respiratorio, Influen..."
123,total_camas,Unidades económicas dedicadas principalmente a...
124,total_camas_publicos,Unidades económicas dedicadas principalmente a...
125,total_camas_privados,Unidades económicas dedicadas principalmente a...


In [ ]:
usadas = pd.DataFrame({'campo': variables_municipales.columns})
usadas

,campo
0,cvegeo
1,nom_ent
2,pt_2015
3,an_2015
4,ne614_015
...,...
69,pct_diabetes
70,total_camas_publicos
71,total_camas_privados
72,total_hospitales_publicos


In [ ]:
diccionario_usadas = diccionario.merge(usadas, on='campo')
diccionario_usadas

,campo,descriptor
0,cvegeo,Clave geografica
1,pt_2015,"población total,2015"
2,an_2015,"15 años y más analfabetas,2015"
3,ne614_015,"6 a 14 años que no asisten a la escuela,2015"
4,bi_2015,"15 años y más con educación básica incompleta,..."
5,sins_15,Porcentaje de Población sin derechohabiencia ...
6,vpt_2015,"Viviendas piso de tierra,2015"
7,vns_2015,"Viviendas sin sanitario,2015"
8,vna_2015,"Viviendas sin Agua,2015"
9,vnd_2015,"Viviendas que no disponen de drenaje,2015"


In [ ]:
diccionario_usadas.to_csv("datos/diccionario_variables_usadas.csv", index=False)